In [ ]:
!pip install orb-models
!pip install --extra-index-url=https://pypi.nvidia.com "cuml-cu12==25.2.*"

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


import os, csv, gc, io, re
import torch
from ase.io import read, write
from ase.constraints import FixAtoms
from ase.optimize import BFGS, QuasiNewton
from ase.mep.neb import NEB
from orb_models.forcefield import pretrained
from orb_models.forcefield.calculator import ORBCalculator

# ───────────────────────────────────────────────────────────────────────────────
# Google‑Drive mount (comment if running locally)
# ───────────────────────────────────────────────────────────────────────────────
from google.colab import drive
drive.mount('/content/drive')


def read_poscar_no_junk(path: str):
    """Read POSCAR/CONTCAR."""
    try:
        return read(path)
    except AssertionError:
        pass

    with open(path) as fh:
        lines = fh.readlines()

    num_line = next(i for i, ln in enumerate(lines)
                    if re.fullmatch(r'\s*\d+(?:\s+\d+)*\s*', ln))
    natoms   = sum(map(int, lines[num_line].split()))
    coord_hdr = next(i for i in range(num_line + 1, len(lines))
                     if lines[i].strip().lower().startswith(('direct', 'cart')))
    last = coord_hdr + 1 + natoms
    return read(io.StringIO(''.join(lines[:last])), format='vasp')

# ───────────────────────────────────────────────────────────────────────────────
# ───────────────────────────────────────────────────────────────────────────────
device = "cuda" if torch.cuda.is_available() else "cpu"
def make_calc():
    return ORBCalculator(pretrained.orb_v3_conservative_inf_omat(
  device=device,
  precision="float32-high",))

# ───────────────────────────────────────────────────────────────────────────────
# 3. Parameters
# ───────────────────────────────────────────────────────────────────────────────
BASE_DIR        = "/content/drive/MyDrive/MLIP-NEB-inputs"
SKIP_SYSTEM     = "#"
IMAGES_BETWEEN  = 7
FMAX            = 0.05
MAX_STEPS       = 1000

# ───────────────────────────────────────────────────────────────────────────────
# 4. Main loop
# ───────────────────────────────────────────────────────────────────────────────
for root, dirs, _ in os.walk(BASE_DIR):
    if not ({"initial", "final"} <= set(dirs)):
        continue

    system = os.path.basename(root)
    if system == SKIP_SYSTEM:
        print(f"{system} (user skip)")
        continue

    save_dir = os.path.join(root, "ORBV3_run")
    csv_path = os.path.join(save_dir, "energies.csv")
    if os.path.exists(csv_path):
        print(f"{system} (already done)")
        continue

    print(f"\n Processing {system}")
    os.makedirs(save_dir, exist_ok=True)

    # ── read endpoints ────────────────────────────────────────────────────
    initial = read_poscar_no_junk(os.path.join(root, "initial", "POSCAR"))
    final   = read_poscar_no_junk(os.path.join(root, "final",   "POSCAR"))

    # freeze framework atoms (tag > 1) and attach calculators
    for atoms in (initial, final):
        atoms.set_constraint(FixAtoms([a.tag > 1 for a in atoms]))
        atoms.calc = make_calc()

    # ── relax endpoints ───────────────────────────────────────────────────
    for atoms, lbl in ((initial, "initial"), (final, "final")):
        traj = os.path.join(save_dir, f"{lbl}_relaxed.traj")
        BFGS(atoms, trajectory=traj).run(fmax=FMAX, steps=MAX_STEPS)
        write(os.path.join(save_dir, f"relaxed_{lbl}.vasp"), atoms, "vasp")

    # reload relaxed structures
    initial = read(os.path.join(save_dir, "initial_relaxed.traj"))
    final   = read(os.path.join(save_dir, "final_relaxed.traj"))
    fixed   = FixAtoms([a.tag > 1 for a in final])

    # ── build NEB band ─────────────────
    images = [initial]
    for _ in range(IMAGES_BETWEEN):
        img = initial.copy()
        img.set_constraint(fixed)
        img.calc = make_calc()
        images.append(img)
    images.append(final)

    # final already has a calc, ensure it's independent
    final.calc = make_calc()

    # NEB 
    neb = NEB(images, parallel=True, k=5, method="eb")
    neb.interpolate("idpp")
    BFGS(neb, trajectory=os.path.join(save_dir, "neb.traj")).run(
        fmax=FMAX, steps=MAX_STEPS
    )

    # ── energies & migration barrier ─────────────────────────────────────
    energies = []
    with open(csv_path, "w", newline="") as fh:
        wr = csv.writer(fh); wr.writerow(["Image", "Energy (eV)"])
        for i, img in enumerate(images):
            try:
                e = img.get_potential_energy()
            except Exception as err:
                print(f"⚠️  {system} img{i}: {err}")
                e = None
            energies.append(e)
            write(os.path.join(save_dir, f"neb_bfgs_{i}.vasp"), img, "vasp")
            wr.writerow([i, e])

    if None not in energies:
        barrier = max(energies) - min(energies)
        with open(csv_path, "a", newline="") as fh:
            csv.writer(fh).writerow([])
            csv.writer(fh).writerow(["Migration Barrier", barrier])
        print(f"{system}: barrier = {barrier:.4f} eV")
    else:
        print(f"{system}: barrier skipped (missing energies)")

    # tidy GPU memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

print("\n All NEB calculations complete!")